In [1]:
!git clone https://github.com/meituan/YOLOv6.git ./utility/YOLOv6
%cd ./utility/YOLOv6

Cloning into './utility/YOLOv6'...
remote: Enumerating objects: 2543, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 2543 (delta 0), reused 1 (delta 0), pack-reused 2540
Receiving objects: 100% (2543/2543), 32.91 MiB | 12.02 MiB/s, done.
Resolving deltas: 100% (1406/1406), done.
/home/hectorandac/Documents/Homeworks/RockPaperScissor/utility/YOLOv6


In [5]:
!python3 ./tools/infer.py --weights ../../weights/yolo6_l+hands_dataset.pt --source ../../raw_hands --yaml ../../raw_hands/data.yaml --img-size 128 128 --save-txt

Namespace(weights='../../weights/yolo6_l+hands_dataset.pt', source='../../raw_hands', yaml='../../raw_hands/data.yaml', img_size=[128, 128], conf_thres=0.4, iou_thres=0.45, max_det=1000, device='0', save_txt=True, not_save_img=False, save_dir=None, view_img=False, classes=None, agnostic_nms=False, project='runs/inference', name='exp', hide_labels=False, hide_conf=False, half=False)
Save directory already existed
Loading checkpoint from ../../weights/yolo6_l+hands_dataset.pt

Fusing model...
Switch model to deploy modality.
/home/hectorandac/.local/lib/python3.10/site-packages/torch/functional.py:478: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2895.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
100%|███████████████████████████████████████| 1988/1988 [07:03<00:00,  4.69it/s]
Results saved to runs/inference/exp


In [20]:
def getBounds(line):
    cl, cX, cY, w, h, confi = line.split()
    cX = float(cX)
    cY = float(cY)
    w = float(w)
    h = float(h)
    confi = float(confi)

    bLeft = cX - (w/2)
    bRight = cX + (w/2)
    bTop = cY - (h/2)
    bBottom = cY + (h/2)
    return [bLeft, bRight, bTop, bBottom, confi]

def average(lst):
    return sum(lst) / len(lst)

In [21]:

import os

directory = './runs/inference/exp/raw_hands'

for root, dirs, files in os.walk(directory):
    for filename in files:
        if filename.endswith(".txt"):
            f = os.path.join(root, filename)
            imageCount = f.replace(".txt", '.jpg')
            chClass = 0

            if "rock" in filename:
                chClass = 0
            elif "paper" in filename:
                chClass = 1
            else:
                chClass = 2

            
            opened = open(f, "r")
            bLefts = []
            bRights = []
            bTops = []
            bBottoms = []
            confidences = []

            lines = opened.readlines()
            for line in lines:
                results = getBounds(line)
                bLefts.append(results[0])
                bRights.append(results[1])
                bTops.append(results[2])
                bBottoms.append(results[3])
                confidences.append(results[4])
            
            opened.close()

            bL = min(bLefts)
            bR = max(bRights)
            bT = min(bTops)
            bB = max(bBottoms)

            wNew = bR - bL
            hNew = bB - bT
            cXNew = bL + (wNew/2)
            cYNew = bT + (hNew/2)
            confidence = average(confidences)

            toWrite = open(f, "w")
            toWrite.write(str(chClass) + " " + str(cXNew) + " " + str(cYNew) + " " + str(wNew) + " " + str(hNew) + " " + str(confidence))
            toWrite.close()


        


In [29]:
import shutil
import glob, os

def move_to_train(expected_file_group, src_image_path, txt_dest, img_dest):
    files = glob.glob(expected_file_group + "/*.txt")
    fNum = len(files)
    
    trainN = int(0.6 * fNum)
    testN = int(0.3 * fNum)
    valN = int(0.1 * fNum)
    for i in range(fNum):
        file_name_txt = os.path.basename(files[i])
        file_name_img = file_name_txt.replace(".txt", '.jpg')

        section = "train"
        if i >= trainN:
            section = "train"
        elif i  >= testN:
            section = "test"
        elif i >= valN:
            section = "val"

        fromImage = os.path.join(src_image_path, file_name_img)
        fromText = os.path.join(expected_file_group, file_name_txt)

        toImage = os.path.join(img_dest, section, file_name_img)
        toText = os.path.join(txt_dest, section, file_name_txt)

        shutil.copyfile(fromImage, toImage)
        shutil.copyfile(fromText, toText)

In [35]:
move_to_train("./runs/inference/exp/raw_hands/rock", "../../raw_hands/rock", "../../input_v3/labels", "../../input_v3/images")
move_to_train("./runs/inference/exp/raw_hands/paper", "../../raw_hands/paper", "../../input_v3/labels", "../../input_v3/images")
move_to_train("./runs/inference/exp/raw_hands/scissors", "../../raw_hands/scissors", "../../input_v3/labels", "../../input_v3/images")

In [36]:
def remove_not_requiered(directory):
    for filename in os.listdir(directory):
        if filename.endswith(".txt"):
            f = os.path.join(directory, filename)
            opened = open(f, "r")

            lines = opened.readlines()
            for line in lines:
                cl, cX, cY, w, h, confi = line.split()
                
            toWrite = open(f, "w")
            toWrite.write(cl + " " + cX + " " + cY + " " + w + " " + h)
            toWrite.close()
            

In [37]:
remove_not_requiered("../../input_v3/labels/test")
remove_not_requiered("../../input_v3/labels/train")
remove_not_requiered("../../input_v3/labels/val")

In [42]:
!mkdir weights
!cp ../../weights/yolo6_l+hands_dataset.pt weights/yolov6l.pt

In [43]:
!python3 ./tools/train.py --img-size 320 --batch 32 --epochs 100 --conf ./configs/yolov6l_finetune.py --data ../../input_v3/data.yaml

Using 1 GPU for training... 
training args are: Namespace(data_path='../../input_v3/data.yaml', conf_file='./configs/yolov6l_finetune.py', img_size=320, batch_size=32, epochs=100, workers=8, device='0', eval_interval=20, eval_final_only=False, heavy_eval_range=50, check_images=False, check_labels=False, output_dir='./runs/train', name='exp', dist_url='env://', gpu_count=0, local_rank=-1, resume=False, write_trainbatch_tb=False, stop_aug_last_n_epoch=15, save_ckpt_on_last_n_epoch=-1, distill=False, distill_feat=False, quant=False, calib=False, teacher_model_path=None, temperature=20, rank=-1, world_size=1, save_dir='runs/train/exp2')

Train: Final numbers of valid images: 987/ labels: 987. 
0.0s for dataset initialization.
Val: Checking formats of labels with 8 process(es): 
393 label(s) found, 0 label(s) missing, 0 label(s) empty, 0 invalid label files:
Convert to COCO format
100%|█████████████████████████████████████| 393/393 [00:00<00:00, 158572.53it/s]
Convert to COCO format finishe